<a href="https://colab.research.google.com/github/The237/DeepLearningCourses/blob/main/02_01_Recreer_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# La boîte générique

In [2]:
class Box():
  def __init__(self):
    pass

  def forward(self, inputs):
    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):
    assert derivee_output.shape == self.output.shape, f'La dérivée output reçue a un shape {derivee_output.shape} et différent du shape de output : {self.output.shape}'

    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f'La dérivée input calculée a un shape {self.derivee_input.shape} et différent du shape de input : {self.inputs.shape}'

    return self.derivee_inputs

  def operation(self):
    pass

  def gradient(self,derivee_output):
    pass

 # La boîte paramétrée

In [3]:
class BoxParams():
  def __init__(self, param):
    self.param = param
    pass

  def forward(self, inputs):
    self.inputs = inputs
    self.output = self.operation()

    return self.output

  def backward(self, derivee_output):

    assert derivee_output.shape == self.output.shape, f'La dérivée output reçue a un shape {derivee_output.shape} et différent du shape de output : {self.output.shape}'

    self.derivee_inputs = self.gradient(derivee_output)
    assert self.derivee_inputs.shape == self.inputs.shape, f'La dérivée input calculée a un shape {self.derivee_input.shape} et différent du shape de input : {self.inputs.shape}'

    self.derivee_param = self.gradient_param(derivee_output)
    assert self.derivee_param.shape == self.param.shape, f'La dérivée de param a un shape {self.derivee_param.shape} et différent du shape de param : {self.param.shape}'

    return self.derivee_inputs

  def operation(self):
    pass

  def gradient(self,derivee_output):
    pass

  def gradient_param(self,derivee_output):
    pass

# La Classe Dot

In [4]:
class Dot(BoxParams):

  def __init__(self, weights):
    super().__init__(weights)

  def operation(self):
    return np.dot(self.inputs, self.param)

  def gradient(self, derivee_output):
     return np.dot(derivee_output,self.param.T)

  def gradient_param(self, derivee_output):
     return np.dot(self.inputs.T, derivee_output)

  def __repr__(self):
    return "Dot Product"

In [5]:
X = np.array([
    [2,3,-2],
    [4,5,-1],
    [-5,2,3],
    [0,5,4]
])

In [6]:
W = np.array([
    [.4967],
    [-.1383],
    [.6477]
])

In [7]:
np.dot(X,W)

array([[-0.7169],
       [ 0.6476],
       [-0.817 ],
       [ 1.8993]])

In [8]:
M = Dot(weights=W)

In [9]:
out = M.forward(X)
out

array([[-0.7169],
       [ 0.6476],
       [-0.817 ],
       [ 1.8993]])

In [10]:
d_out = np.random.randn(4,1)
d_out

array([[-0.31115614],
       [-0.46739056],
       [-0.16049706],
       [-0.52744829]])

In [11]:
M.backward(d_out)

array([[-0.15455126,  0.04303289, -0.20153583],
       [-0.23215289,  0.06464011, -0.30272887],
       [-0.07971889,  0.02219674, -0.10395395],
       [-0.26198357,  0.0729461 , -0.34162826]])

# La Classe Add

In [12]:
class Add(BoxParams):

  def __init__(self, biais):
    super().__init__(biais)

  def operation(self):
    return self.inputs + self.param

  def gradient(self, derivee_output):
     return np.ones_like(self.inputs)*derivee_output

  def gradient_param(self, derivee_output):
      r = np.ones_like(self.param)*derivee_output
      return r.sum(axis = 0).reshape(1, self.param.shape[1])

  def __repr__(self):
    return "Add Biais"

In [13]:
B = np.random.randn(1,1)

In [14]:
B

array([[1.17129915]])

In [15]:
b = Add(biais = B)
b

Add Biais

In [16]:
out_b = b.forward(out)
out_b

array([[0.45439915],
       [1.81889915],
       [0.35429915],
       [3.07059915]])

In [17]:
d_out

array([[-0.31115614],
       [-0.46739056],
       [-0.16049706],
       [-0.52744829]])

In [18]:
b.backward(d_out)

array([[-0.31115614],
       [-0.46739056],
       [-0.16049706],
       [-0.52744829]])

# La Classe Sigmoid

In [86]:
class Sigmoid(Box):
  def __init__(self):
     super().__init__()

  def operation(self):
    return 1 /(1 + np.exp(-1 * self.inputs))

  def gradient(self,derivee_output):
    return self.output * (1 - self.output) * derivee_output

  def __repr__(self):
    return "Sigmoid"

In [20]:
sig = Sigmoid()

In [21]:
sig_out = sig.forward(out_b)
sig_out

array([[0.61168466],
       [0.86043398],
       [0.58765972],
       [0.95566357]])

In [22]:
sig.backward(d_out)

array([[-0.07390784],
       [-0.05612769],
       [-0.03889097],
       [-0.02234836]])

# La Classe Loss

In [46]:
class Loss():
  def __init__(self):
    pass

  def forward(self, prediction, target):
    assert prediction.shape == target.shape
    self.prediction = prediction
    self.target = target
    loss = np.mean((self.target -self.prediction)**2)
    return loss

  def backward(self):
    self.loss_derivee = -2 * (self.target - self.prediction)
    assert self.loss_derivee.shape == self.prediction.shape, f'La dérivée loss a un shape {self.loss_derivee.shape} et différent du shape de Prediction : {self.prediction.shape}'

    return self.loss_derivee

In [24]:
Y = np.random.randn(4,1)
Y

array([[-1.44202839],
       [-1.08312131],
       [ 0.30588018],
       [-1.61360313]])

In [25]:
P = sig_out

In [47]:
mse = Loss()

In [48]:
mse.forward(P, Y)

3.66891888678465

In [49]:
mse.backward()

array([[4.1074261 ],
       [3.88711058],
       [0.56355909],
       [5.13853339]])

# La classe Dense

In [87]:
class Dense():

  def __init__(self, neurons, activation = None):
    self.neurons = neurons
    self.activation = activation
    self.params = []
    self.suite = []
    self.initialization = True

  def build(self, inputs):
    # weights initialization
    np.random.seed(42)
    self.weights = np.random.randn(inputs.shape[1], self.neurons)
    self.biais = np.random.randn(1, self.neurons)
    self.params.append(self.weights)
    self.params.append(self.biais)

    # construction de la suite d'opérations
    self.suite = [Dot(weights= self.params[0]), Add(biais=self.params[1])]
    if self.activation:
      self.suite.append(self.activation)

  def forward(self, inputs):
    if self.initialization:
      self.build(inputs)
      self.initialization = False

      for boite in  self.suite:
        inputs = boite.forward(inputs)

    self.output = inputs

    return self.output

  def backward(self, derivee_output):
    assert derivee_output.shape == self.output.shape

    for box in reversed(self.suite):
      derivee_output = box.backward(derivee_output)
    derivee_inputs = derivee_output

    return derivee_inputs

  def __repr__(self):
    r = f'Dense Layer(neurons = {self.neurons})'
    if self.activation:
      r +=" avec Sigmoid"
    return r

In [88]:
sigmoid = Sigmoid()
couche = Dense(neurons = 2, activation = sigmoid)
couche

Dense Layer(neurons = 2) avec Sigmoid

In [75]:
X

array([[ 2,  3, -2],
       [ 4,  5, -1],
       [-5,  2,  3],
       [ 0,  5,  4]])

In [89]:
couche.forward(X)

array([[0.99320003, 0.99604286],
       [0.99912347, 0.99968533],
       [0.42276305, 0.97817014],
       [0.97978765, 0.99941659]])

In [90]:
couche.params

[array([[ 0.49671415, -0.1382643 ],
        [ 0.64768854,  1.52302986],
        [-0.23415337, -0.23413696]]),
 array([[1.57921282, 0.76743473]])]

In [91]:
d_out = np.random.randn(4,2)
d_out

array([[-0.46947439,  0.54256004],
       [-0.46341769, -0.46572975],
       [ 0.24196227, -1.91328024],
       [-1.72491783, -0.56228753]])

In [92]:
couche.backward(d_out)

array([[-0.00187061,  0.00120335,  0.00024173],
       [-0.00018133, -0.00048599,  0.00012933],
       [ 0.03497832, -0.02397904, -0.00426045],
       [-0.0169224 , -0.02262434,  0.00807543]])

In [93]:
couche.suite

[Dot Product, Add Biais, Sigmoid]

# La classe Model

In [106]:
class Model():

  def __init__(self, layers):
    self.layers = layers

  def forward(self, inputs):
    for layer in self.layers:
      inputs = layer.forward(inputs)
    self.output = inputs
    return self.output


  def backward(self, loss_derivee):
    assert loss_derivee.shape == self.output.shape

    for layer in reversed(self.layers):
      loss_derivee = layer.backward(loss_derivee)

    return loss_derivee

  def __repr__(self):
    r = "Layers ................."
    for layer in self.layers:
      r+=f" \n {str(layer)}"
    return r

In [107]:
model = Model(layers = [
    Dense(neurons = 3, activation = sigmoid),
    Dense(neurons = 1)
    ])

In [108]:
model

Layers ................. 
 Dense Layer(neurons = 3) avec Sigmoid 
 Dense Layer(neurons = 1)

In [101]:
Y

array([[-1.44202839],
       [-1.08312131],
       [ 0.30588018],
       [-1.61360313]])

In [109]:
model.forward(X)

array([[2.47005633],
       [2.53479834],
       [1.89807891],
       [1.92678186]])

In [110]:
loss_derivee = np.random.randn(4,1)
loss_derivee

array([[-0.23415337],
       [-0.23413696],
       [ 1.57921282],
       [ 0.76743473]])

In [111]:
model.backward(loss_derivee)

array([[-0.02166009, -0.00198156,  0.00600408],
       [-0.01558369,  0.00516281,  0.01226937],
       [ 0.00662474,  0.0076687 , -0.01500028],
       [ 0.01131573,  0.00059134, -0.01919523]])

# Update des paramètres